In [ ]:
import pandas as pd
import numpy as np

## 读取行业数据
industry_data = pd.read_csv('data_1.0\\INDUSTRY_GICS.csv', index_col=0, )
industry_data.head()

# 保存数据
industry_data.to_csv('data_1.0\\labeled_industry.csv')

##市值分组转数字标签
df = pd.DataFrame()

# 读取市值数据
cap = pd.read_csv(
    'data_1.0\\MKT_CAP_ARD.csv',
    index_col=0
)
cap.head()

# 读取交易量数据，用于排除停牌股票，交易量为0则停牌
volume = pd.read_csv(
    'data_1.0\\VOLUME.csv',
    index_col=0
)
volume.head()

# 将市值对数化
lcap = np.log(cap)
lcap.tail()

# 识别交易量为NAN值或0为股票停牌，设置对应时间股票市值数据为nan不参与后续分组，机器学习模型里面特征一般不允许是Nan
lcap[((volume.isna())|(volume==0))] = np.nan
# lcap.dropna(axis=0, inplace=True, how='all')

# 保存2013年以后的数据
lcap = lcap.loc['2013-01-04':]
lcap.head()

# 将股票按照市值分成5组，每组中股票数量相等
numerical_labeled_lcap = lcap.apply(pd.qcut, axis=1, q=5, labels=False)
numerical_labeled_lcap.tail()

# 保存数据
numerical_labeled_lcap.to_csv('data_1.0\\numerical_labeled_lcap.csv')

##动量计算转数字标签
# 读取收盘价数据
close = pd.read_csv(
    'data_1.0\\CLOSE.csv',
    index_col=0
)
close.head()

# 读取交易量数据
volume = pd.read_csv(
    'data_1.0\\VOLUME.csv',
    index_col=0
)
close.tail()

# 识别交易量为NAN值或0为股票停牌，设置对应时间股票市值数据为nan不参与后续分组        # 注意pandas需要升级版本，否则没有isna()函数；
close[(volume.isna()) | (volume==0)] = np.nan

# 计算5日动能
# mom = close.pct_change(periods=5)
# mom.tail()

mom = close/close.shift(5)-1

# 去除缺失值
mom.dropna(axis=0,inplace=True, how='all')

mom = mom.loc['2013-01-04':]
mom.head(20)

# 将股票按照动能分成5组，每组中股票数量相等, 按行去做；
numerical_labeled_mom = mom.apply(pd.qcut, axis=1, q=5, labels=False)
numerical_labeled_mom.tail()

numerical_labeled_mom.to_csv('data_1.0\\numerical_labeled_mom.csv')

##非停牌股票日收益率计算
# 读取收盘价数据
close = pd.read_csv(
    'data_1.0\\CLOSE.csv',
    index_col=0
)
close.head()

# 读取交易量数据
volume = pd.read_csv(
    'data_1.0\\VOLUME.csv',
    index_col=0
)
volume.tail()

# 识别交易量为NAN值或0为股票停牌，设置对应时间股票市值数据为nan不参与后续分组
close[(volume.isna()) | (volume==0)] = np.nan

# 计算日收益率
close.sort_index(inplace=True)
pct_change = close.pct_change()
pct_change.tail()

# 取出2013年以后的数据
pct_change = pct_change.loc['2013-01-04':]
pct_change.head()

# 保存数据
pct_change.to_csv('data_1.0\\pct_change.csv')

##非停牌股票日收益率转数字标签
# 读取收盘价数据
close = pd.read_csv(
    'data_1.0\\CLOSE.csv',
    index_col=0
)
close.head()

# 读取交易量数据
volume = pd.read_csv(
    'data_1.0\\VOLUME.csv',
    index_col=0
)
volume.tail()

# # 识别交易量为NAN值或0为股票停牌，设置对应时间股票市值数据为nan不参与后续分组
close[(volume.isna()) | (volume==0)] = np.nan

# 计算日收益率
close.sort_index(inplace=True)
pct_change = close.pct_change()
pct_change.tail()

# 去除缺失值
pct_change.dropna(axis=0,inplace=True, how='all')

pct_change = pct_change.loc['2013-01-04':]
pct_change.head()

# 对于上涨的股票标记为1，下跌或涨幅为0的股票标记为0
pct_change[pct_change > 0]=1
pct_change[pct_change <= 0]=0
pct_change.head()

pct_change.to_csv('data_1.0\\labeled_pct_change.csv')

In [ ]:
##数据读取
import pandas as pd
import numpy as np

from sklearn.naive_bayes import BernoulliNB
import warnings; warnings.simplefilter('ignore') #忽略可能会出现的警告信息，警告并不是错误，可以忽略；

start_date = '2013-01-11'
end_date='2017-01-01'

# 读取收益率数据
pct_change = pd.read_csv(
    'data_1.0\\pct_change.csv',
    index_col=0,
    parse_dates=True,
)
pct_change.head()

# 读取市值标签数据
labeled_lcap = pd.read_csv(
    'data_1.0\\numerical_labeled_lcap.csv',
    index_col=0,
    parse_dates=True,
)
labeled_lcap.head()

# 读取动量标签数据
labeled_mom = pd.read_csv(
    'data_1.0\\numerical_labeled_mom.csv',
    index_col=0,
    parse_dates=True,
)
labeled_mom.head()

# 读取行业标签数据
labeled_industry = pd.read_csv(
    'data_1.0\\labeled_industry.csv',
    index_col=0,
    parse_dates=True,
    encoding='utf-8'
)
labeled_industry.head()

# 读取成交量数据，用于制作交易日历；用于在循环的过程中，帮助我们控制具体的日期；usecols指定读取的列
volume = pd.read_csv(
    'data_1.0\\VOLUME.csv',
    index_col=0,
    parse_dates=True,
    usecols=[0,]
)
volume.head()

# 读取收益率标签数据
labeled_pct_change = pd.read_csv(
    'data_1.0\\labeled_pct_change.csv',
    index_col=0,
    parse_dates=True,
)
labeled_pct_change.head()

# 取出回测范围内的交易日期，用于循环去控制日期；
volume.sort_index(inplace=True)
new_volume = volume.loc[start_date:end_date]
trade_calendar = new_volume.index
trade_calendar

##策略核心
# 创建空Series用于储存策略收益率
strategy_return = pd.Series(index=trade_calendar)

# 进行策略回测
# i = 101
for i in range(len(trade_calendar)-2):

    # 获取训练对应x，y数据的时间和预测时x，y数据的时间
    train_x_time = trade_calendar[i]
    train_y_time = trade_calendar[i+1]
    predict_x_time = trade_calendar[i+1]
    predict_y_time = trade_calendar[i+2]

    # 模型训练用数据读取
    train_data = pd.DataFrame(
        {
            'labeled_industry': labeled_industry.loc[train_x_time], #.loc取出行后会返回一个列名为index的ndarray
            'labeled_lcap': labeled_lcap.loc[train_x_time]+200, #要加200是因为伯努利模型要防止不同特征在dummy时取到相同取值
            'labeled_mom': labeled_mom.loc[train_x_time]+300,

            'labeled_pct_change':labeled_pct_change.loc[train_y_time]
        }
    )
    train_data.dropna(axis=0, inplace=True, how='any')
    #train_data

    # 生成训练用哑变量矩阵
    dummy_data1 = pd.get_dummies(train_data['labeled_industry'])
    dummy_data2 = pd.get_dummies(train_data['labeled_lcap'])
    dummy_data3 = pd.get_dummies(train_data['labeled_mom'])

    # 合并哑变量矩阵train_x，将特征拼接起来
    dummy_train_x = pd.concat([dummy_data1, dummy_data2, dummy_data3], axis=1)
    #dummy_train_x.head()

    # 模型预测用数据读取
    predict_data = pd.DataFrame(
        {
            'labeled_industry': labeled_industry.loc[predict_x_time],
            'labeled_lcap': labeled_lcap.loc[predict_x_time]+200,
            'labeled_mom': labeled_mom.loc[predict_x_time]+300,

            'pct_change': pct_change.loc[predict_y_time]
        }
    )
    predict_data.dropna(axis=0, inplace=True, how='any')
    #predict_data.head()
    
    #  生成预测用哑变量矩阵
    dummy_predict_data1 = pd.get_dummies(predict_data['labeled_industry'])
    dummy_predict_data2 = pd.get_dummies(predict_data['labeled_lcap'])
    dummy_predict_data3 = pd.get_dummies(predict_data['labeled_mom'])
    #dummy_predict_data3.head()
    # 合并哑变量矩阵predict_x
    dummy_predict_x = pd.concat([dummy_predict_data1, dummy_predict_data2, dummy_predict_data3], axis=1)
    #dummy_predict_x.head()

    # 由于训练和预测的特征数量可能不同，所以需要columns的并集对列标签进行调整。比如某行业的所有公司在某时刻都处于停牌，下一天有些公司复牌，则在该时刻训练集会少一个行业(被dropna)但测试集没有少，导致训练出问题
    character_union = dummy_train_x.columns.union(dummy_predict_x.columns) #对训练集和测试集的列标签取并集
    dummy_train_x = dummy_train_x.reindex(columns=character_union, fill_value=0) #.reindex能够将索引进行加减，注意不是重命名index或列名，而是参数index或columns指定的列表中同名的会保留，如果列表中有dataframe中没有的，则会新增该index或列，fill_value指定新增的填充值，不指定默认为NaN
    dummy_predict_x = dummy_predict_x.reindex(columns=character_union, fill_value=0)
    
    # 训练模型
    clf = BernoulliNB()
    clf.fit(dummy_train_x.values, train_data['labeled_pct_change'].values) #.values是因为传入sklearn分类器的数据一般要求为ndarray
    
    # 进行预测并保存数据
    prediction = clf.predict(dummy_predict_x.values)
    predict_data['prediction'] = prediction
    #predict_data

# 方法一：
    # 计算预测日策略收益率并保存到Series中
    if predict_data['prediction'].sum() == 0: #防止出现整个策略期内都没有成交信号，prediction全为0，导致计算时分母为0的情况
        strategy_return[predict_y_time] = 0
    else:
        strategy_return[predict_y_time] = np.average(predict_data['pct_change'], weights=predict_data['prediction']) #以持仓为权重的收益率的加权平均，等价于方法二的结果
    # strategy_return.head()
    ### 注意：此方法涉及股票是否停牌的未来数据，对于predict_y_time日停牌的股票，策略回测时选择不持仓

# # 方法二：收益率*持仓/总共购买的股票数
# strategy_return[predict_y_time] = np.sum(predict_data['pct_change'] * predict_data['prediction'])/np.sum(predict_data['prediction'])
# strategy_return.head()
# strategy_return[:5]

##策略收益绘图
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn
import tushare as ts

benchmark = ts.get_k_data('399006','2013-01-01','2017-01-01')
benchmark.index = pd.to_datetime(benchmark['date'])

benchmark = (benchmark['close'].pct_change()+ 1).cumprod()

plt.figure(figsize=(10,6))
plt.plot((strategy_return+1).cumprod(), label='strategy_return')
plt.plot(benchmark, label='benchmark')
plt.yticks()
plt.tit
plt.xticks(rotation=45, )
plt.legend()

##没有考虑交易成本、没有剔除涨停股票